<a href="https://colab.research.google.com/github/allifizzuddin89/Malaria-Prediction/blob/master/Malaria_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly&response_type=code

Enter your authorization code:
4/1AY0e-g4qR1qBffwS7Tdt-NTYZRGrvCfQclbfJS2VeFbKZjkAZ_XBMgWZUaU
Mounted at /content/gdrive


In [6]:
!pwd

/content/drive/MyDrive/COLAB/Malaria detection


In [5]:
import os
os.chdir('/content/drive/MyDrive/COLAB/Malaria detection')

In [ ]:
!unzip "/content/drive/MyDrive/COLAB/Malaria detection/dataset.zip"

Archive:  /content/drive/MyDrive/COLAB/Malaria detection/dataset.zip
   creating: Test/
   creating: Test/Parasite/
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_10.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_11.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_12.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_13.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_14.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_15.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_16.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_17.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_18.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_19.png  
  inflating: Test/Parasite/C39P4thinF_original_IMG_20150622_105554_cell_20

In [ ]:
!mkdir dataset

In [ ]:
!pwd

/content/drive/My Drive/COLAB/Malaria detection


In [ ]:
!mv "/content/drive/MyDrive/COLAB/Malaria detection/Train" "/content/drive/MyDrive/COLAB/Malaria detection/dataset"

In [ ]:
!rm "/content/drive/MyDrive/COLAB/Malaria detection/dataset.zip"

In [7]:
import tensorflow as tf
print(tf.__version__)

2.4.1


In [ ]:
#### Creating CNN Using Scratch Ad Transfer Learning

In [8]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt


In [9]:
 #resize image to 224 px
 Image_size = [224, 224]

#path dir in google drive
#generate test, train data before call here
 train_path = "/content/drive/MyDrive/COLAB/Malaria detection/dataset/Train"
 test_path = "/content/drive/MyDrive/COLAB/Malaria detection/dataset/Test"

In [10]:
vgg19 = VGG19(input_shape=Image_size + [3], weights='imagenet', include_top=False) # [3] is rgb channel

80142336/80134624 [==============================] - 1s 0us/step


In [11]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [12]:
#dont train existing weights since we are using transfer learning
#retraining only happen in last layer

for layer in vgg19.layers:
  layer.trainable = False

In [13]:
#useful to get number of output class
folders = glob("/content/drive/MyDrive/COLAB/Malaria detection/dataset/Train/*") 

In [14]:
folders

['/content/drive/MyDrive/COLAB/Malaria detection/dataset/Train/Parasite',
 '/content/drive/MyDrive/COLAB/Malaria detection/dataset/Train/Uninfected']

In [15]:
#our layers - can add more
x = Flatten()(vgg19.output)

In [16]:
#dense layer
prediction = Dense(len(folders), activation='softmax')(x)

#create a model object
model = Model(inputs=vgg19.input, outputs=prediction)

In [17]:
#structure of the model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     